In [2]:
# Configuration
import os
import sys
REPO_NAME = "Practis"
def get_repo_basepath():
    cd = os.path.abspath(os.curdir)
    return cd[:cd.index(REPO_NAME) + len(REPO_NAME)]
REPO_BASE_PATH = get_repo_basepath()
sys.path.append(REPO_BASE_PATH)
#=============================================================================================
import json
import torch
import numpy as np
from tactis.gluon.estimator_perceiver import TACTiSPTrainingNetwork, TACTiSPPredictionNetwork
from tactis.gluon.dataset import generate_backtesting_datasets
from tactis.gluon.metrics import compute_validation_metrics
from inference_util import create_predictor

# torch.set_float32_matmul_precision("medium")


# Enter variable below ↓

In [6]:
# Enter your model config json file path in below, for example
# MODEL_PARAMS_JSON = "./train_config/elec_model_config.json"
# MODEL_CKPT_PATH = "./train_ckpt/electricity/epoch_100.bin"
MODEL_PARAMS_JSON = 
MODEL_CKPT_PATH = 

In [7]:
# prepare dataset, here I use electricy dataset as an example
history_length = 2
prediction_length = 2
total_length = history_length+prediction_length 
backtest_id = 2

metadata, train_data, test_data = generate_backtesting_datasets("electricity_hourly", backtest_id, total_length - 1)
history_length *= metadata.prediction_length
metadata.prediction_length *=prediction_length
chosen_rows = np.arange(321)
mean = 0.0
std = 1.0

# for entry in train_data:
#     entry["target"] = (entry["target"][chosen_rows, :] - mean)/std
for entry in test_data:
    entry["target"] = (entry["target"][chosen_rows, :] - mean)/std
num_series = train_data[0]["target"].shape[0]

# Run all cells below ↓

In [9]:
# Load model
if os.path.exists(MODEL_PARAMS_JSON):
    with open(MODEL_PARAMS_JSON, "r") as json_file:
        params = json.load(json_file)
model = TACTiSPTrainingNetwork(num_series=num_series, 
                               model_parameters = params["model_parameters"])
model.load_state_dict(torch.load(MODEL_CKPT_PATH))

<All keys matched successfully>

In [10]:
prediction_network = TACTiSPPredictionNetwork(
    num_series=num_series,
    model_parameters=params["model_parameters"],
    prediction_length=metadata.prediction_length,
    num_parallel_samples=100,
    ).to("cuda")
predictor = create_predictor(train_network = model, 
                             prediction_network = prediction_network, 
                             num_series = num_series, 
                             history_length = history_length,
                             prediction_length = metadata.prediction_length,
                             )

In [11]:
metrics = compute_validation_metrics(
    predictor=predictor,
    dataset=test_data,
    window_length=history_length + metadata.prediction_length,
    num_samples=100,
    split=False,
)

/scratch/jl986/Practis/tactis/model/decoder_perceiver.py:106: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  half = (mask_l + mask_r)//2


In [12]:
#epochs = 100
metrics

{'CRPS': 0.061206292496671734,
 'ND': 0.07862516385842264,
 'NRMSE': 0.3668832328473297,
 'MSE': 610026.6552208106,
 'CRPS-Sum': 0.01996823281138207,
 'ND-Sum': 0.025692053282689784,
 'NRMSE-Sum': 0.03272503979693871,
 'MSE-Sum': 500107678.4761905,
 'Energy': 68090.13283076297,
 'Conditional-RMSE': 781.0420237459332,
 'Conditional-NRMSE': 0.10865255098814651,
 'num_nan': 0,
 'num_inf': 0}

# Run multiple

In [20]:
ls = [compute_validation_metrics(
    predictor=predictor,
    dataset=test_data,
    window_length=history_length + metadata.prediction_length,
    num_samples=100,
    split=False,
) for _ in range(10)]

/scratch/jl986/Practis/tactis/model/decoder_perceiver.py:106: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  half = (mask_l + mask_r)//2
/scratch/jl986/Practis/tactis/model/decoder_perceiver.py:106: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  half = (mask_l + mask_r)//2
/scratch/jl986/Practis/tactis/model/decoder_perceiver.py:106: Us

In [27]:
sum(list(map(lambda x: x["CRPS-Sum"], ls))) / 10

0.020045556372755718